In [ ]:
import json
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

data_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\search_20241106-223705_sodium+ion+battery+anode-sodium+ion+battery+cathode-sodium+ion+battery+electrode_annotated_rephrased_once.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

texts = []
labels = []

for i in data:
    texts.append(i["text"])
    labels.append(i["label_int"])

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42
)

train_dataset = Dataset.from_dict({
    'text': train_texts,
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'text': val_texts,
    'labels': val_labels
})

raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})
print(len(texts))

200


In [2]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 160
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 40
    })
})


In [3]:
model_name = "batterydata/batterybert-cased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, torch_dtype="auto")

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [6]:
import evaluate

metric = evaluate.load("accuracy")

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  eval_strategy="epoch",
                                  num_train_epochs=7)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)


In [10]:
trainer.train()

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.43410539627075195, 'eval_accuracy': 0.85, 'eval_runtime': 0.3574, 'eval_samples_per_second': 111.922, 'eval_steps_per_second': 13.99, 'epoch': 1.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.36275264620780945, 'eval_accuracy': 0.85, 'eval_runtime': 0.3246, 'eval_samples_per_second': 123.21, 'eval_steps_per_second': 15.401, 'epoch': 2.0}


...

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.22679421305656433, 'eval_accuracy': 0.95, 'eval_runtime': 0.3343, 'eval_samples_per_second': 119.638, 'eval_steps_per_second': 14.955, 'epoch': 6.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.2334994375705719, 'eval_accuracy': 0.95, 'eval_runtime': 0.3137, 'eval_samples_per_second': 127.521, 'eval_steps_per_second': 15.94, 'epoch': 7.0}
{'train_runtime': 32.6269, 'train_samples_per_second': 34.327, 'train_steps_per_second': 4.291, 'train_loss': 0.5064248221261161, 'epoch': 7.0}


TrainOutput(global_step=140, training_loss=0.5064248221261161, metrics={'train_runtime': 32.6269, 'train_samples_per_second': 34.327, 'train_steps_per_second': 4.291, 'total_flos': 294684382003200.0, 'train_loss': 0.5064248221261161, 'epoch': 7.0})

In [11]:
trainer.evaluate()

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.2334994375705719,
 'eval_accuracy': 0.95,
 'eval_runtime': 0.3137,
 'eval_samples_per_second': 127.524,
 'eval_steps_per_second': 15.94,
 'epoch': 7.0}

In [12]:
trainer.save_model("batterybert-cased-abstract_finetuned_200")

In [14]:
from transformers import pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [17]:
text = r"MXene nanosheets and ordered porous carbons both have their own advantages and disadvantages. Assembling and combining the advantages of the two will be a good choice for battery electrode hosts of active materials. In this work, an electrostatic separation-adsorption strategy is proposed to realize the ordered alternating self-assembly of MXene nanosheets and ordered porous carbon (MPOC), obtaining a unique wall-like porous material with a high conductivity and interconnected porous nanostructure, which strengthens the transfer rate of electrons and ions simultaneously. Meanwhile, the introduction of N-doping from porous carbon into MPOC prolongs the cycle life. When use red phosphorus (RP) as active materials, the MPOC@RP anode exhibited high-capacity output (2454.3 and 2408.1 mAh g-1 in lithium-ion batteries (LIBs) and sodium-ion batteries (SIBs) at 0.1 C) and long cycle life (the decay rates per cycle of 0.028% and 0.036% after 1500 and 1200 cycles at 2 C in LIBs and SIBs respectively). The successful application in RP anodes displays great potential in other electrode materials such as silicon, sulfur, selenium, and so on. Meanwhile, this strategy is also effective to design other composites materials like MXene and carbon nanotubes, MXene and Graphene, and so on."

In [18]:
model_out = AutoModelForSequenceClassification.from_pretrained("batterybert-cased-abstract_finetuned_200")
with torch.no_grad():
    logits = model_out(text).logits
predicted_class_id = logits.argmax().item()
print(predicted_class_id)

TypeError: string indices must be integers

In [15]:
classifier(r"MXene nanosheets and ordered porous carbons both have their own advantages and disadvantages. Assembling and combining the advantages of the two will be a good choice for battery electrode hosts of active materials. In this work, an electrostatic separation-adsorption strategy is proposed to realize the ordered alternating self-assembly of MXene nanosheets and ordered porous carbon (MPOC), obtaining a unique wall-like porous material with a high conductivity and interconnected porous nanostructure, which strengthens the transfer rate of electrons and ions simultaneously. Meanwhile, the introduction of N-doping from porous carbon into MPOC prolongs the cycle life. When use red phosphorus (RP) as active materials, the MPOC@RP anode exhibited high-capacity output (2454.3 and 2408.1 mAh g-1 in lithium-ion batteries (LIBs) and sodium-ion batteries (SIBs) at 0.1 C) and long cycle life (the decay rates per cycle of 0.028% and 0.036% after 1500 and 1200 cycles at 2 C in LIBs and SIBs respectively). The successful application in RP anodes displays great potential in other electrode materials such as silicon, sulfur, selenium, and so on. Meanwhile, this strategy is also effective to design other composites materials like MXene and carbon nanotubes, MXene and Graphene, and so on.")

[{'label': 'non-battery', 'score': 0.995262861251831}]

In [16]:
classifier(r"Highly crystalline CuFeS2 containing earth-abundant and environmentally friendly elements prepared via a high-temperature synthesis exhibits an excellent electrochemical performance as an anode material in sodium-ion batteries. The initial specific capacity of 460 mAh g-1 increases to 512 mAh g-1 in the 150th cycle and then decreases to a still very high value of 444 mAh g-1 at 0.5 A g-1 in the remaining 550 cycles. Even for a large current density, a pronounced cycling stability is observed. Here, we demonstrate that combining the results of X-ray powder diffraction experiments, pair distribution function analysis, and 23Na NMR and Mössbauer spectroscopy investigations performed at different stages of discharging and charging processes allows elucidation of very complex reaction mechanisms. In the first step after uptake of 1 Na/CuFeS2, nanocrystalline NaCuFeS2 is formed as an intermediate phase, which surprisingly could be recovered during charging. On increasing the Na content, Cu+ is reduced to nanocrystalline Cu, while nanocrystalline Na2S and nanosized elemental Fe are formed in the discharged state. After charging, the main crystalline phase is NaCuFeS2. At the 150th cycle, the mechanisms clearly changed, and in the charged state, nanocrystalline CuxS phases are observed. At later stages of cycling, the mechanisms are altered again: NaF, Cu2S, and Cu7.2S4 appeared in the discharged state, while NaF and Cu5FeS4 are observed in the charged state. In contrast to a typical conversion reaction, nanocrystalline phases play the dominant role, which are responsible for the high reversible capacity and long-term stability.")

[{'label': 'non-battery', 'score': 0.9952206015586853}]